In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
#载入数据集
mnist = input_data.read_data_sets("./MNIST_data",one_hot = True)#one_hot是把标签转化成为向量的形式
##比如数据对应的是3通过one_hot表示即为[0001000000]

#每个批次的大小 一次性放入的图片的数量
batch_size = 100

#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size #整除

#定义两个placeholder占位符
x = tf.placeholder(tf.float32,[None,784])#行先不管（批次100则过会儿为100） 列为784的向量,MNIST数据集的维度是28*28=784
y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)#keep_prob 表示为多少 比例的神经元 参与工作
lr = tf.Variable(0.001, dtype=tf.float32)#学习率

# #创建一个简单的神经网络 不考虑隐藏层
# W = tf.Variable(tf.zeros([784,10]))#初始化为0 不太好
# b = tf.Variable(tf.zeros([10]))
# prediction  = tf.nn.softmax(tf.matmul(x,W)+b)

#创建一个 神经网络(不需要这么多层 设置多层以产生过拟合 可以体现出 dropout失活的效果 )
W1 = tf.Variable(tf.truncated_normal([784,500],stddev=0.1))#截断的正态分布函数
b1 = tf.Variable(tf.zeros([500])+0.1)
L1 = tf.nn.tanh(tf.matmul(x,W1)+b1)
L1_drop = tf.nn.dropout(L1,keep_prob)#keep_prob 表示为多少 比例的神经元 参与工作

W2 = tf.Variable(tf.truncated_normal([500,300],stddev=0.1))
b2 = tf.Variable(tf.zeros([300])+0.1)
L2 = tf.nn.tanh(tf.matmul(L1_drop,W2)+b2)
L2_drop = tf.nn.dropout(L2,keep_prob)#keep_prob 表示为多少 比例的神经元 参与工作

W3 = tf.Variable(tf.truncated_normal([300,10],stddev=0.1))
b3 = tf.Variable(tf.zeros([10])+0.1)
prediction  = tf.nn.softmax(tf.matmul(L2_drop,W3)+b3)


# #二次代价函数
# loss = tf.reduce_mean(tf.square(y - prediction))

# #softmax+交叉熵
# loss = tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction)

#softmax+交叉熵 平均值
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))

#梯度下降法
train_step = tf.train.AdamOptimizer(lr).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction , 1))
## tf.argmax(y,1) 表示 找到一维向量中 值为1（1最大）的那一个值所在的位置 【y为真实值】
## tf.argmax(prediction,1) 表示 找到找到一维向量中 数值最大的那一个值所在的位置
###判断后两个值 是否相等 相等即为true 不相等就是 false

#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
##tf.cast() 转化布尔类型为float32 TRUE 转为1 false 转为0
###然后求一下平均值 比如 【1111111110】 准确率为0.9

with tf.Session() as sess:
    #变量初始化
    sess.run(init)
    for epoch in range(51):#所有循环一共进行21次
        sess.run(tf.assign(lr,0.001 * (0.095 ** epoch)))
        for batch in range(n_batch):#内部循环代表把所有图片训练一次
            ##n_batch 代表一共有多少个批次
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            ##每次循环 获取100张图片 batch_xs 保存图片数据 batch_ys 保存图片的标签
            sess.run(train_step,feed_dict = {x:batch_xs,y:batch_ys,keep_prob:1.0})
            
        learning_rate = sess.run(lr)    
        test_acc = sess.run(accuracy,feed_dict = {x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})   #测试数据
     
        print("Inter"+str(epoch)+"Testing accuracy "+str(test_acc)+",Learning rate="+str(learning_rate))

Extracting ./MNIST_data\train-images-idx3-ubyte.gz
Extracting ./MNIST_data\train-labels-idx1-ubyte.gz
Extracting ./MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data\t10k-labels-idx1-ubyte.gz
Inter0Testing accuracy 0.9512,Learning rate=0.001
Inter1Testing accuracy 0.9603,Learning rate=9.5e-05
Inter2Testing accuracy 0.9614,Learning rate=9.025e-06
Inter3Testing accuracy 0.9614,Learning rate=8.57375e-07
Inter4Testing accuracy 0.9614,Learning rate=8.14506e-08
